In [139]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import random
import math
from scipy.stats import ttest_ind
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqUtils.ProtParam import ProteinAnalysis as Prot
from Bio.SeqUtils.IsoelectricPoint import IsoelectricPoint as IP

In [147]:
def character_pos(sequence,character):
    s = sequence
    c = character
    positions = [pos for pos, char in enumerate(s) if char == c]
    return(positions)

In [98]:
#Version with amount of sequences normalization
def position_matrix(df,windowsize,character):
    position_dict = {}
    position_dict_norm = {}
    length = 0
    for candidate in df['protein']:
        percentages = []
        positions = character_pos(candidate,character)
        length = length+len(positions)
        for x in positions:
            for i in range(1,windowsize+1): 
                if x-i < 0:
                    continue
                name = candidate[x-i] + '_'+str(i)
                if name in position_dict:
                    position_dict[name] += 1
                else:
                    position_dict[name] = 1
            for t in range(1,windowsize+1):
                if x+t >= len(candidate):
                    continue
                name = candidate[x+t] + '_'+str(t)
                if name in position_dict:
                    position_dict[name] += 1
                else:
                    position_dict[name] = 1
    #Normalize position values by length of dataframe
    for lel in position_dict:
        position_dict_norm[lel] = position_dict[lel] / length
    return(position_dict_norm,position_dict)

In [212]:
#Version with amino acid occurence normalization
def position_matrix(df,windowsize,character,quartile):
    position_dict = {}
    position_dict_norm = {}
    length = 0
    for candidate in df['protein']:
        percentages = []
        positions = character_pos(candidate,character)
        length = length+len(positions)
        for x in positions:
            for i in range(1,windowsize+1): 
                if x-i < 0:
                    continue
                name = candidate[x-i] + '_'+str(i)
                if name in position_dict:
                    position_dict[name] += 1
                else:
                    position_dict[name] = 1
            for t in range(1,windowsize+1):
                if x+t >= len(candidate):
                    continue
                name = candidate[x+t] + '_'+str(t)
                if name in position_dict:
                    position_dict[name] += 1
                else:
                    position_dict[name] = 1
    #Normalize position values by length of dataframe
    for lel in position_dict:
        position_dict_norm[lel] = position_dict[lel] / (quartile[lel[:-2]])
    return(position_dict_norm)

In [196]:
#Version with amino acid occurence normalization
def position_matrix(df,windowsize,character,quartile):
    position_dict = {}
    position_dict_norm = {}
    length = 0
    for candidate in df['protein']:
        percentages = []
        positions = character_pos(candidate,character)
        length = length+len(positions)
        for x in positions:
            for i in range(1,windowsize+1): 
                if x-i < 0:
                    continue
                name = candidate[x-i] + '_'+str(i)
                if name in position_dict:
                    position_dict[name] += 1
                else:
                    position_dict[name] = 1
            for t in range(1,windowsize+1):
                if x+t >= len(candidate):
                    continue
                name = candidate[x+t] + '_'+str(t)
                if name in position_dict:
                    position_dict[name] += 1
                else:
                    position_dict[name] = 1
    #Normalize position values by length of dataframe
    for lel in position_dict:
        position_dict_norm[lel] = position_dict[lel] /math.sqrt(quartile[lel[:-2]]*quartile[character])
        break
    return(position_dict_norm)

# Quartile analysis

In [213]:
df = pd.read_excel('Tables/SI Table4.xlsx',skiprows=2)


In [214]:
amino_acids = ['A', 'R', 'N', 'D', 'C', 'E', 'Q', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']

In [215]:
raw_count=[]
percentages = []
for my_seq in df['protein']:
    analysed_seq = Prot(my_seq)
    percentages.append(analysed_seq.get_amino_acids_percent())
    raw_count.append(analysed_seq.count_amino_acids())
df_percentage = pd.DataFrame(percentages)
df_raw_count = pd.DataFrame(raw_count)

In [216]:
df1_per = df_percentage[:20*4388]
df2_per =df_percentage[20*4388:20*2*4388]
df3_per =df_percentage[20*2*4388:20*3*4388]
df4_per =df_percentage[20*3*4388:]
df5_per = df_percentage[20*(-155):]

In [217]:
sum_dict1 = df_raw_count[:4388].sum().to_dict()
sum_dict2 = df_raw_count[4388:2*4388].sum().to_dict()
sum_dict3 = df_raw_count[2*4388:3*4388].sum().to_dict()
sum_dict4 = df_raw_count[3*4388:].sum().to_dict()


In [218]:
df_plot = pd.concat([df1_per,df2_per,df3_per,df4_per])

In [219]:
aminos = list(df_percentage.columns)
change = []
for x in df_plot.iterrows():
    count=0
    for z in x[1]:
        change.append([z,aminos[count]])
        count+=1

In [220]:
df_plot1 = pd.DataFrame(change,columns=['Frequency','Amino'])

In [221]:
df1_per = df_plot1[:20*4388]
df2_per =df_plot1[20*4388:20*2*4388]
df3_per =df_plot1[20*2*4388:20*3*4388]
df4_per =df_plot1[20*3*4388:]

In [222]:
df1_per['Hue'] = '1st_q'
df2_per['Hue'] = '2nd_q'
df3_per['Hue'] = '3rd_q'
df4_per['Hue'] = '4th_q'

/var/folders/xb/cc3hd4q54k900w4vqym76dq00000gp/T/ipykernel_60112/1303841559.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_per['Hue'] = '1st_q'
/var/folders/xb/cc3hd4q54k900w4vqym76dq00000gp/T/ipykernel_60112/1303841559.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_per['Hue'] = '2nd_q'
/var/folders/xb/cc3hd4q54k900w4vqym76dq00000gp/T/ipykernel_60112/1303841559.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [223]:
df1 = df[:4388]
df2 =df[4388:2*4388]
df3 =df[2*4388:3*4388]
df4 =df[3*4388:]

In [224]:
# Next three cells build position matrix for all tiles in first quartile, the code repeats for every quartile and runs quickly
dataframes = {}
df_names = []
for x in amino_acids:
    df_names.append(x+'_position1')
for name in df_names:
    dataframes[name] = position_matrix(df1,9,name[0],sum_dict1)

In [225]:
position_df = {}
for key in dataframes.keys():
    step1 = sorted(dataframes[key].items()) # sorted by key, return a list of tuples
    position_df[key] = pd.DataFrame(step1,columns=['Aminoacid_pos','counts'])
    position_df[key]['Hue'] = key[0]

In [226]:
df_position_all1 = pd.concat(position_df.values())

In [227]:
dataframes2 = {}
df_names2 = []
for x in amino_acids:
    df_names2.append(x+'_position1')
for name in df_names2:
    dataframes2[name] = position_matrix(df2,9,name[0],sum_dict1)

In [228]:
position_df2 = {}
for key in dataframes2.keys():
    step1 = sorted(dataframes2[key].items()) # sorted by key, return a list of tuples
    position_df2[key] = pd.DataFrame(step1,columns=['Aminoacid_pos','counts'])
    position_df2[key]['Hue'] = key[0]

In [229]:
df_position_all2 = pd.concat(position_df2.values())

In [230]:
dataframes3 = {}
df_names3 = []
for x in amino_acids:
    df_names3.append(x+'_position1')
for name in df_names3:
    dataframes3[name] = position_matrix(df3,9,name[0],sum_dict1)

In [231]:
position_df3 = {}
for key in dataframes3.keys():
    step1 = sorted(dataframes3[key].items()) # sorted by key, return a list of tuples
    position_df3[key] = pd.DataFrame(step1,columns=['Aminoacid_pos','counts'])
    position_df3[key]['Hue'] = key[0]

In [232]:
df_position_all3 = pd.concat(position_df3.values())

In [233]:
dataframes4 = {}
df_names4 = []
for x in amino_acids:
    df_names4.append(x+'_position1')
for name in df_names4:
    dataframes4[name] = position_matrix(df4,9,name[0],sum_dict1)

In [234]:
position_df4 = {}
for key in dataframes4.keys():
    step1 = sorted(dataframes4[key].items()) # sorted by key, return a list of tuples
    position_df4[key] = pd.DataFrame(step1,columns=['Aminoacid_pos','counts'])
    position_df4[key]['Hue'] = key[0]

In [235]:
df_position_all4 = pd.concat(position_df4.values())

In [236]:
position_list =[['A_1','A_2','A_3','A_4','A_5','A_6','A_7','A_8','A_9'],
['C_1','C_2','C_3','C_4','C_5','C_6','C_7','C_8','C_9'],
 ['D_1','D_2','D_3','D_4','D_5','D_6','D_7','D_8','D_9'],
 ['E_1','E_2','E_3','E_4','E_5','E_6','E_7','E_8','E_9'],
 ['F_1','F_2','F_3','F_4','F_5','F_6','F_7','F_8','F_9'],
 ['G_1','G_2','G_3','G_4','G_5','G_6','G_7','G_8','G_9'],
 ['H_1','H_2','H_3','H_4','H_5','H_6','H_7','H_8','H_9'],
 ['I_1','I_2','I_3','I_4','I_5','I_6','I_7','I_8','I_9'],
 ['K_1','K_2','K_3','K_4','K_5','K_6','K_7','K_8','K_9'],
 ['L_1','L_2','L_3','L_4','L_5','L_6','L_7','L_8','L_9'],
 ['M_1','M_2','M_3','M_4','M_5','M_6','M_7','M_8','M_9'],
 ['N_1','N_2','N_3','N_4','N_5','N_6','N_7','N_8','N_9'],
 ['P_1','P_2','P_3','P_4','P_5','P_6','P_7','P_8','P_9'],
 ['Q_1','Q_2','Q_3','Q_4','Q_5','Q_6','Q_7','Q_8','Q_9'],
 ['R_1','R_2','R_3','R_4','R_5','R_6','R_7','R_8','R_9'],
 ['S_1','S_2','S_3','S_4','S_5','S_6','S_7','S_8','S_9'],
 ['T_1','T_2','T_3','T_4','T_5','T_6','T_7','T_8','T_9'],
 ['V_1','V_2','V_3','V_4','V_5','V_6','V_7','V_8','V_9'],
 ['W_1','W_2','W_3','W_4','W_5','W_6','W_7','W_8','W_9'],
['Y_1','Y_2','Y_3','Y_4','Y_5','Y_6','Y_7','Y_8','Y_9']]

ylim_list2 = [[0.01,0.16],[0,0.10],[0.005,0.28],[0.05,0.3],[0.0,0.27],[0.025,0.25],[0.0,0.09],[0.02,0.19],[0.01,0.15],[0.025,0.3],[0.01,0.14],[0.03,0.17],[0.01,0.165],[0.015,0.125],[0.015,0.125],[0.025,0.25],[0.02,0.14],[0.02,0.2],[0,0.125],[0.015,0.15]]

In [238]:
import os
quartile = 1
    # Define the figure size
a4_dims = (5, 5)
for x in [df_position_all1,df_position_all2,df_position_all3,df_position_all4]:
    count = 0
    
    # Create a 'Figures' directory if it doesn't already exist
    if not os.path.exists('Test'):
        os.makedirs('Test')
    df_pivot = x.pivot('Aminoacid_pos', 'Hue', 'counts')
    # Loop over the slices of the dataframe that are multiples of 9
    for i in range(0, len(df_pivot), 9):
        # Select the slice of the dataframe
        df_slice = df_pivot.iloc[i:i+9, :]

        # Create the heatmap
        plt.figure(figsize=(15,10))
        g = sns.heatmap(df_slice,cmap='viridis',vmin=ylim_list2[count][0],vmax=ylim_list2[count][1])
        #g = sns.heatmap(df_slice,cmap='viridis')
        #g.set(ylim=(ylim_list[count][0],ylim_list[count][1]))
        # Save the figure
        plt.savefig(f'SI Data 2/new_norm1_heatmap_{i}_'+str(quartile)+'.pdf')
        # Close the figure to avoid memory leaks
        plt.close()
        plt.clf()
        count+=1
    quartile +=1

/var/folders/xb/cc3hd4q54k900w4vqym76dq00000gp/T/ipykernel_60112/3734531687.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  df_pivot = x.pivot('Aminoacid_pos', 'Hue', 'counts')
/var/folders/xb/cc3hd4q54k900w4vqym76dq00000gp/T/ipykernel_60112/3734531687.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  df_pivot = x.pivot('Aminoacid_pos', 'Hue', 'counts')
/var/folders/xb/cc3hd4q54k900w4vqym76dq00000gp/T/ipykernel_60112/3734531687.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  df_pivot = x.pivot('Aminoacid_pos', 'Hue', 'counts')
/var/folders/xb/cc3hd4q54k900w4vqym76dq00000gp/T/ipykernel_60112/3734531687.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  df_pivot = x.pivot('Aminoacid_pos', 'Hue', 'counts')


<Figure size 640x480 with 0 Axes>